Рекомендательные системмы являются одними из самых популярных приложений DS. Они используются для прогнозирования предпочтений,  которые пользователь поставит какому-нибудь товару в магазине. Amazon использует их, чтобы предлагать продукты клиентам, YouTube решает какое видео воспроизводить следующим, а Facebook рекомендует посты. Существуют также рекомендательные системы для таких доменов, как рестораны, фильмы и онлайн-знакомства. Более того, для некоторых компаний, таких как Netflix, бизнес-модель и ее успех зависят от эффективности их рекомендаций. Netflix даже предложил миллион долларов в 2009 году каждому, кто сможет улучшить его систему рекомендаций на 10%.

Вообще говоря, рекомендательные системы могут быть поделены на 3 типа:
* Базовый подход: предлагайте общие рекомендации каждому пользователю в зависимости от популярности фильма и / или жанра. Например, Кинопоиск 250.
* Content-based рекомендательные системы. Эта система использует метаданные элемента, например жанр, режиссер, описание, актеры и т.д. для фильмов. Общая идея, лежащая в основе этих рекомендательных систем, заключается в том, что если человеку нравится конкретный предмет, ему также понравится предмет, похожий на него. Алгоритм будет использовать метаданные прошлых элементов пользователя. Хорошим примером может служить YouTube, где на основе вашей истории он предлагает вам новые видео, которые вы потенциально можете посмотреть.
* Коллаборативная фильтрация. Эти системы пытаются предсказать рейтинг, который пользователь поставит элементу, на основе прошлых оценок и предпочтений других пользователей. Коллаборативной фильтрации не требуются метаданные элемента.

# Простые рекомендации

* Сначала нужно определить метрику, по которой будем сортировать фильмы
* Затем посчитать метрику для каждого фильма
* Отсортировать фильмы, и выбрать топ результатов

Для работы будем использовать популярный датасет MovieLens. Датасет можно скачать по [ссылке](https://www.kaggle.com/rounakbanik/the-movies-dataset). 

In [1]:
import pandas as pd
import numpy as np

metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Одна из базовых метрик - это рейтинг. Однако он не принимает во внимание популярность фильма. Таким образом, фильм с рейтингом 9 от  10 пользователей будет считаться «лучше», чем фильм с рейтингом 8,9 от 10 000 пользователей. Поэтому стоит использовать взвешенный рейтинг - учитывающий средний рейтинг и количество голосов.

<img src="weighted.png">

* v - количество голосов за фильм

* m - минимальное количество голосов, необходимое для внесения в таблицу

* R - средний рейтинг фильма

* C - средний голос по всем фильмам

m - гиперпараметр. Выберем для него 90 перцентиль - то есть для попадания в таблицу, у фильма должно быть больше голосов, чем у 90 процентов других фильмов.

In [2]:
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


In [3]:
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [4]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [5]:
def weighted_rating(x, m=m, C=C):
    ###Реализуйте функцию, которая считает взвешенный рейтинг элемента###
    
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [6]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [7]:
q_movies = q_movies.sort_values('score', ascending=False)

In [8]:
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Ну вот, мы построили простую рейтинговую таблицу на основании взвешенного рейтинга фильмов. Смахивает на кинопоиск, правда?

# Content-based рекомендации

Идея content-based подхода заключается в том, чтобы по истории действий пользователя создать для него вектор его предпочтений в пространстве предметов и рекомендовать товары, близкие к этому вектору.

Попробуем рекомендовать фильмы, похожие на конкретный фильм. Для этого вычислим попарные косинусные расстояния для всех фильмов на основе их сюжетных описаний и порекомендуем фильмы на основе этой метрики.

<img src="content.png">

In [9]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

Невозможно вычислить сходство между любыми двумя обзорами в их необработанном виде, поэтому вычислим векторы слов. Как следует из названия, векторы слов - это векторизованное представление слов в документе. Векторы несут семантическое значение. Например, мужчина и король будут иметь векторные представления близко друг к другу, в то время как мужчина и женщина будут иметь представления далеко друг от друга.

Мы вычислим Term Frequency-Inverse Document Frequency (TF-IDF) для каждого документа. Это даст нам матрицу, в которой каждый столбец представляет слово в обзорном словаре (все слова, которые встречаются хотя бы в одном документе), а каждый столбец представляет фильм.

По сути, оценка TF-IDF - это частота встречаемости слова в документе, взвешенная с уменьшением числа документов, в которых оно встречается. Это сделано для уменьшения важности слов, которые часто встречаются в обзорах сюжетов, и, следовательно, их значимости при вычислении окончательной оценки сходства.

TfIdfVectorizer уже реализован в библиотеке scikit. Таким образом, нам нужно проделать следующие шаги:

* импортировать Tfidf из scikit-learn
* убрать стоп-слова, так как они не несут полезной информации
* Замените пропущенные значения пустой строкой
* построить матрицу TF-IDF


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 5),min_df=0, stop_words='english') #Определите объект TF-IDF Vectorizer, задайте параметр, который уберет стоп-слова из английского языка

metadata['overview'] = metadata['overview'].fillna('')#Замените все пропущенные значения в колонке пустой строкой

tfidf_matrix = tfidf.fit_transform(metadata['overview']) #Постройте матрицу TF-IDF, вызвав метод fit-transform 

tfidf_matrix.shape

(45466, 4565384)

In [11]:
tfidf.get_feature_names()[5000:5010]

['13 wives issue',
 '13 wives issue local',
 '13 wives issue local authorities',
 '13 wounding',
 '13 year',
 '13 year imprisonment',
 '13 year imprisonment kidnap',
 '13 year imprisonment kidnap murder',
 '13 year old',
 '13 year old american']

Таким образом, у нас получается 75827 различных слов в нашем наборе данных. С помощью этой матрицы мы можем вычислить оценку сходства между двумя фильмами. Мы будем использовать cosine similarity.

<img src="cos.png">

Поскольку мы использовали TF-IDF, вычисление скалярного произведения между каждым вектором напрямую даст вам оценку косинусного сходства. Следовательно, можно использовать linear_kernel() sklearn вместо cosine_similarities(), поскольку он быстрее. Мы получим матрицу размера 45466x45466, где каждый фильм будет вектором-столбцом 1x45466.

In [12]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
cosine_sim.shape

(45466, 45466)

In [14]:
cosine_sim[1]

array([0.00192322, 1.        , 0.00475309, ..., 0.        , 0.0026321 ,
       0.0010098 ])

Нужно определить функцию, которая принимает на вход название фильма и выводит список из 10 наиболее похожих фильмов. Нужен механизм для определения индекса фильма в metadata по его названию.

In [15]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [16]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

Функция должна выполнять следующие действия:

* Получить индекс фильма по его названию.

* Получить список оценок косинусного сходства для этого конкретного фильма со всеми фильмами. Преобразуйте его в список кортежей, где первый элемент - это его позиция, а второй - оценка сходства.

* Отсортируйте вышеупомянутый список кортежей на основе оценок сходства (второй элемент).

* Получите 10 лучших элементов этого списка. Игнорируйте первый элемент, так как он относится к себе (фильм, наиболее похожий на себя - это он сам).

* Верните заголовки, соответствующие индексам верхних элементов.

In [17]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(6)
    return qualified

In [18]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title] #Получите индекс по названию

    sim_scores = list(enumerate(cosine_sim[idx])) # Список оценок сходства для фильма по его индексу из матрицы оценок

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #Отсортируйте массив по скорам (второй элемент)

    sim_scores = sim_scores[1:11] # Возьмите 10 первых элементов (кроме самого фильма)

    movie_indices = [i[0] for i in sim_scores] #Получите массив индексов этих 10 элементов 

    # Верните названия топ10 похожих фильмов
    return metadata['title'].iloc[movie_indices]

In [19]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
585                                                 Batman
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
9230                    Batman Beyond: Return of the Joker
21194    Batman Unmasked: The Psychology of the Dark Kn...
35983                                    Batman: Bad Blood
18035                                     Batman: Year One
11753                                            Slow Burn
Name: title, dtype: object

# Коллаборативная фильтрация

<img src="collaborative.png">

Помимо метаданных фильмов, у нас есть еще один ценный источник информации: данные о рейтингах пользователей. Наша система рекомендаций может порекомендовать фильм, похожий на «Начало (2010)», на основе оценок пользователей. Другими словами, какие еще фильмы получили аналогичные оценки других пользователей? Это был бы пример коллаборативной фильтрации item-item. Таким примером является рекомендация по типу «Пользователям, которым понравился этот элемент, понравились и другие». Мы будем исследовать набор данных ratings.csv, и сформируем векторы оценок пользователей.

В файле links.csv лежит маппинг разных id фильмов. В частности в ratings.csv (который мы будем использовать) используется movieId. В уже знакомом нам metadata используется tmdbId. Чтобы использовать обе таблички, добавим в metadata колонку с movieId - которая по сути является маппингом на колонку metadata['id'] с помощью links.

In [20]:
links = pd.read_csv('links.csv')

links = links.dropna()

In [21]:
links = links.drop_duplicates(subset=["tmdbId"],keep='first')
id_to_movieId = dict(links[['tmdbId','movieId']].values)

In [22]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               45466 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [23]:
def to_numeric(x):
    try:
        return(int(x))
    except(ValueError):
        return None

In [24]:
# metadata['id'] = pd.to_numeric(metadata['id'],errors='foo')#errors='skip')
metadata['id'] = metadata['id'].apply(to_numeric)

In [25]:
metadata = metadata.dropna(subset=['id'])

metadata['movieId'] = metadata['id'].map(id_to_movieId)

In [26]:
#Создадим словарь отображения названия фильма в его movieId
title_to_id = dict(zip(metadata.title.tolist(), metadata.movieId.tolist()))

In [27]:
title_to_id

{'Toy Story': 1.0,
 'Jumanji': 2.0,
 'Grumpier Old Men': 3.0,
 'Waiting to Exhale': 4.0,
 'Father of the Bride Part II': 5.0,
 'Heat': 131274.0,
 'Sabrina': 915.0,
 'Tom and Huck': 8.0,
 'Sudden Death': 9.0,
 'GoldenEye': 10.0,
 'The American President': 11.0,
 'Dracula: Dead and Loving It': 12.0,
 'Balto': 13.0,
 'Nixon': 14.0,
 'Cutthroat Island': 15.0,
 'Casino': 16.0,
 'Sense and Sensibility': 165321.0,
 'Four Rooms': 18.0,
 'Ace Ventura: When Nature Calls': 19.0,
 'Money Train': 20.0,
 'Get Shorty': 21.0,
 'Copycat': 22.0,
 'Assassins': 23.0,
 'Powder': 24.0,
 'Leaving Las Vegas': 25.0,
 'Othello': 103683.0,
 'Now and Then': 27.0,
 'Persuasion': 164805.0,
 'The City of Lost Children': 29.0,
 'Shanghai Triad': 30.0,
 'Dangerous Minds': 31.0,
 'Twelve Monkeys': 32.0,
 'Wings of Courage': 33.0,
 'Babe': 34.0,
 'Carrington': 35.0,
 'Dead Man Walking': 36.0,
 'Across the Sea of Time': 37.0,
 'It Takes Two': 131582.0,
 'Clueless': 39.0,
 'Cry, the Beloved Country': 123244.0,
 'Richard I

В вашем распоряжении есть два файла - ratings.csv и ratings_small.csv. Второй файлик содержит намного меньше информации, но для быстрой работы давайте использовать его (хотя качество, конечно же, получится хуже). При желании вы можете обучить модель на полном файле рейтингов (ratings.csv).

In [28]:
ratings = pd.read_csv('ratings_small.csv')

In [29]:
ratings.tail(3)

,userId,movieId,rating,timestamp
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [30]:
ratings.shape

(100004, 4)

In [31]:
a = ratings['userId'].value_counts()<50
a

547    False
564    False
624    False
15     False
73     False
       ...  
221     True
444     True
484     True
35      True
485     True
Name: userId, Length: 671, dtype: bool

In [32]:
for i in a.index:
    print(i)

547
564
624
15
73
452
468
380
311
30
294
509
580
213
212
472
388
23
457
518
461
232
102
262
475
306
119
654
358
529
575
105
56
353
664
48
587
165
596
195
384
463
605
481
665
607
19
285
199
150
405
268
242
505
615
480
514
130
299
17
423
574
111
346
157
187
128
407
402
77
382
598
134
355
537
243
430
534
313
585
95
561
608
427
239
220
247
460
312
577
387
292
431
544
652
78
439
648
176
88
287
426
200
562
522
500
295
149
240
270
442
177
236
303
408
22
373
412
152
367
471
466
558
214
345
125
57
205
394
253
185
4
501
275
597
363
553
41
433
94
627
584
434
599
72
86
118
283
99
342
34
265
133
344
189
26
620
328
646
418
350
61
428
595
533
21
510
371
83
519
81
93
563
250
235
520
390
487
324
647
626
91
516
159
378
75
255
659
254
396
641
316
609
417
297
486
548
33
582
120
525
219
550
527
592
148
531
182
178
602
559
97
309
656
570
528
282
523
496
245
175
68
92
201
386
110
441
450
362
483
251
168
84
8
671
234
263
603
391
502
169
304
38
381
43
493
536
85
416
572
655
339
497
217
36
623
257
67
560
5
188


In [33]:
ratings['userId'].value_counts().values[428:]

array([49, 49, 49, 49, 48, 48, 48, 48, 47, 46, 46, 46, 46, 46, 46, 46, 46,
       46, 45, 45, 45, 45, 44, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 42,
       42, 42, 42, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 40, 40, 40, 39,
       39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 38,
       38, 38, 38, 38, 38, 37, 37, 37, 37, 36, 36, 36, 36, 36, 35, 35, 35,
       34, 34, 34, 34, 34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 32, 32, 32,
       32, 32, 32, 32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30,
       30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 28, 28, 28, 27, 27, 27,
       27, 27, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26, 26, 26, 26, 26, 26,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 24, 24, 24,
       24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 22,
       22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20

In [34]:
for count,i in enumerate(ratings['userId'].value_counts().values>=50):
#     print(count)
    if(i==False):
        print(count)

427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670


In [35]:
more50marks = list(ratings['userId'].value_counts().index)[0:427]
drop_userid = list(ratings['userId'].value_counts().index)[428:]

In [36]:
drop_userid.sort()
# drop_userid

In [37]:
more50marks # эти id нужно оставить а остальные удалить

[547,
 564,
 624,
 15,
 73,
 452,
 468,
 380,
 311,
 30,
 294,
 509,
 580,
 213,
 212,
 472,
 388,
 23,
 457,
 518,
 461,
 232,
 102,
 262,
 475,
 306,
 119,
 654,
 358,
 529,
 575,
 105,
 56,
 353,
 664,
 48,
 587,
 165,
 596,
 195,
 384,
 463,
 605,
 481,
 665,
 607,
 19,
 285,
 199,
 150,
 405,
 268,
 242,
 505,
 615,
 480,
 514,
 130,
 299,
 17,
 423,
 574,
 111,
 346,
 157,
 187,
 128,
 407,
 402,
 77,
 382,
 598,
 134,
 355,
 537,
 243,
 430,
 534,
 313,
 585,
 95,
 561,
 608,
 427,
 239,
 220,
 247,
 460,
 312,
 577,
 387,
 292,
 431,
 544,
 652,
 78,
 439,
 648,
 176,
 88,
 287,
 426,
 200,
 562,
 522,
 500,
 295,
 149,
 240,
 270,
 442,
 177,
 236,
 303,
 408,
 22,
 373,
 412,
 152,
 367,
 471,
 466,
 558,
 214,
 345,
 125,
 57,
 205,
 394,
 253,
 185,
 4,
 501,
 275,
 597,
 363,
 553,
 41,
 433,
 94,
 627,
 584,
 434,
 599,
 72,
 86,
 118,
 283,
 99,
 342,
 34,
 265,
 133,
 344,
 189,
 26,
 620,
 328,
 646,
 418,
 350,
 61,
 428,
 595,
 533,
 21,
 510,
 371,
 83,
 519,
 81,
 

In [38]:
 #Отфильтруйте таблицу - оставьте только те userId, которые проставили не менее 50 оценок
ratings_f = ratings[ratings['userId'].isin(more50marks)]

In [39]:
ratings_f

,userId,movieId,rating,timestamp
20,2,10,4.0,835355493
21,2,17,5.0,835355681
22,2,39,5.0,835355604
23,2,47,4.0,835355552
24,2,50,4.0,835355586
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [40]:
#Создайте pivot таблицу из отфильтрованной ratings_f. 
# Индексы - фильмы, колонки - пользователи, значения - рейтинг
ratings_pivot = pd.pivot_table(ratings_f,index=["movieId"],columns = 'userId',values=["rating"])
ratings_pivot.head()

rating                                          ...                   \
userId     2   3   4    5    7   8   12   13   15  17   ...  655 656 658 659   
movieId                                                 ...                    
1          NaN NaN NaN  NaN  3.0 NaN NaN  5.0  2.0 NaN  ...  NaN NaN NaN NaN   
2          NaN NaN NaN  NaN  NaN NaN NaN  NaN  2.0 NaN  ...  4.0 NaN NaN NaN   
3          NaN NaN NaN  4.0  NaN NaN NaN  NaN  NaN NaN  ...  NaN NaN NaN NaN   
4          NaN NaN NaN  NaN  NaN NaN NaN  NaN  NaN NaN  ...  NaN NaN NaN NaN   
5          NaN NaN NaN  NaN  NaN NaN NaN  NaN  4.5 NaN  ...  NaN NaN NaN NaN   

                                      
userId   660  662  664  665 667  671  
movieId                               
1        2.5  NaN  3.5  NaN NaN  5.0  
2        NaN  5.0  NaN  3.0 NaN  NaN  
3        NaN  NaN  NaN  3.0 NaN  NaN  
4        NaN  NaN  NaN  NaN NaN  NaN  
5        NaN  NaN  NaN  3.0 NaN  NaN  

[5 rows x 427 columns]

##### Matrix Factorization.

Что ж, теперь в нашем распоряжении есть матрица оценок пользователь-айтем. Из математики мы знаем, что любую матрицу можно разложить на произведение трех матриц - например алгоритмом SVD. Но матрицы оценок очень разрежены, 99 % — обычное дело. А SVD не знает, что такое пропуски. Заполнять их средним значением не очень хочется. И в целом, нас не очень интересует матрица сингулярных значений — мы просто хотим получить скрытое представление пользователей и предметов, которое при перемножении будет приближать истинный рейтинг. Можно сразу раскладывать на две матрицы.

<img src="svd.png">

Что же делать с пропусками? Забить на них. Оказалось, что можно успешно обучать приближать рейтинги по метрике RMSE с помощью SGD или ALS, вообще игнорируя пропуски. Первый такой алгоритм — Funk SVD, который придумали в 2006 году в ходе решения соревнования от Netflix.

Но например в задаче рекомендации товаров, мы имеем уже не матрицу оценок, а матрицу некоторых событий. Она будет состоять в основном из нулей и единичек, иногда каких то чисел побольше. Таким образом, у  нас будут присутствовать только положительные примеры. У нас нет примеров товаров, которые человек никогда не купит - получается мы не можем понять, человек не видел товар или он ему не нравится. Таким образом фидбэк от пользователя может быть двух типов:

* Explicit feedback - есть положительные и отрицательные примеры.
* Implicit feedback - есть только положительные.

Так вот, забить на пропуски получается только в случае задачи explicit feedback. В случае implicit можно заполнить пропущенные значения например нулем, и настроить веса в оптимизируемом функционале - низкие для нулей, и повыше для ненулевых ячеек.

Для того чтобы лучше разобраться в математике, можно прочитать например вот [этот пост](https://habr.com/ru/company/yandex/blog/241455/).

<img src="netflix.png">

Алгоритм SVD реализован в библиотеке [Surprise](https://github.com/NicolasHug/Surprise). Далее мы обучим модель и используем ее для прогнозирования рейтингов фильмов, которые данный пользователь, например с 𝑖𝑑 = 2, еще не получил оценку.

In [41]:
!pip install scikit-surprise

In [42]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_f[['userId','movieId','rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)

accuracy.rmse(predictions)

RMSE: 0.8988


0.8988282793021625

Напишем нашу функцию, которая принимает на вход id пользователя, а на выходе предлагает ему топ 10 фильмов, которые он еще не видел.

In [60]:
def pred_user_rating(ui):
    if ui in ratings_f['userId'].unique():
         #Из таблицы ratings_f создайте list фильмов, которые оценил конкретный пользователь
        ui_list = list(ratings_f[ratings_f['userId']==ui]['movieId'])
        
         #Создайте инвертированный словарь title_to_id, но только для тех фильмов, которые ui не видел. То есть исключите
        #из списка фильмов множество ui_list. Инвертированный - то есть ключи стали значениями, а значения - ключами
        
        id_to_title = {}
        for i in title_to_id.items():
            id_to_title.update({i[1] : i[0]})

        for i in ui_list:
            try:
                del id_to_title[i]
            except(KeyError):
                pass
        
        d = id_to_title
        
        #С помощью нашей обученной модели, проставим предсказанные рейтинги фильмам, которые пользователь еще не видел.
        predictedL = []
        for i, j in d.items():     
            predicted = algorithm.predict(ui, i)
            predictedL.append((j, predicted[3])) 
        #Создайте датафрейм из массива predictedL с колонками ['movies', 'ratings']
        pdf = pd.DataFrame(predictedL,columns = ['movies','ratings'])
        
        #Отсортируйте таблицу по колонке ratings, от большего - к меньшему 
        pdf = pdf.sort_values(by=['ratings'],  ascending=False)
        
        pdf.set_index('movies', inplace=True)    
        return pdf.head(10)        
    else:
        print("Пользователь не найден в списке!")
        return None

In [61]:
user_id = 2
predicted_ratings = pred_user_rating(user_id)

In [62]:
predicted_ratings

,ratings
movies,
The Godfather,4.511773
The African Queen,4.462973
Chinatown,4.444383
The Departed,4.435921
The Shawshank Redemption,4.401778
The Philadelphia Story,4.377169
Kids,4.376169
Toy Story 3,4.367572
To Kill a Mockingbird,4.354270
